<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_style_transfer_musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL DEPENDENCIES

In [2]:
!python -m pip install setuptools wheel
!python -m pip install -U audiocraft
!pip install ffmpeg-python
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write

model = MusicGen.get_pretrained('facebook/musicgen-melody')

import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Check if librosa is installed, if not then install it
try:
    import librosa
except Exception:
    !pip install librosa
    import librosa

# # Check if mido is installed, if not then install it
# try:
#     import mido
# except Exception:
#     !pip install mido
#     import mido

#Check if madmom is installed, if not then install it
try:
    import madmom
except Exception:
    !pip install "packaging==23.1"  # Adjust packaging to the required version
    !pip install madmom  # Ensure madmom is installed

#Check if pyaudio is installed, if not then install it
try:
    import pyaudio
except Exception:
    !apt-get install portaudio19-dev
    !pip install pyaudio
    import pyaudio

#Check if BeatNet is installed, if not then install it
try:
    from BeatNet.BeatNet import BeatNet
except Exception:
    !pip install BeatNet
    from BeatNet.BeatNet import BeatNet

#Check if ruberband is installed, if not then install it
try:
    import pyrubberband as pyrb
except ImportError:
    !sudo apt-get install -y rubberband-cli
    !pip install pyrubberband
    import pyrubberband as pyrb
    import soundfile as sf

import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Check if runes-client is installed, if not then install it
try:
    import runes_client
except ImportError:
    !pip install runes-client
    import runes_client

# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if BeatNet is installed, if not then install it
try:
    from BeatNet.BeatNet import BeatNet
except Exception:
    !pip install BeatNet
    from BeatNet.BeatNet import BeatNet

#Check if ruberband is installed, if not then install it
try:
    import pyrubberband as pyrb
except ImportError:
    !sudo apt-get install -y rubberband-cli
    !pip install pyrubberband
    import pyrubberband as pyrb


import librosa
import soundfile as sf

import uuid
import os
import runes_client.core as runes
from runes_client import RunesFilePath, utils, ui_param
import random
import os
import shutil
import uuid
import numpy as np

placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "af9ac502-4b0d-45a1-a789-01684f8d917a" #@param {type:"string"}
runes_token = RUNES_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

random_descriptions = ["Harmonious Melody","Syncopated Rhythms","Soaring Vocals","Jazzy Improvisation","Deep Basslines","Ethereal Harmonics","Funky Grooves","Classical Elegance","Mellow Acoustics","Pounding Drums","Twinkling Synthesizers","Soulful Crooning","Raw Blues","Uplifting Choruses","Heavy Metal","Ambient Textures","Reggae Vibes","Electronic Beats","Vibrant Salsa","Lush Orchestration","Folk Simplicity","Rock Anthems","Pop Hooks","Gospel Choirs","Latin Rhythms","Punk Energy","Swing Dancing","Disco Funk","Tribal Drums","Dreamy Ballads","Baroque Complexity","New Age Serenity","Country Twang","Bluegrass Banjo","Operatic Drama","Ragtime Piano","Celtic Flutes","Hip-Hop Flow","Glitchy Electronics","Ska Horns","Indie Experimentation","Psychedelic Sounds","Flamenco Guitar","Bollywood Fusion","Smooth Jazz","Grunge Distortion","Techno Pulse","Dubstep Wobbles","Afrobeat Rhythms","Lounge Chillout","Whispering Worry","Laughing Joy","Sobbing Sadness","Thundering Anger","Chirping Contentment","Humming Thoughtfulness","Shouting Frustration","Sighing Relief","Ringing Surprise","Clapping Excitement","Tapping Impatience","Roaring Fear","Murmuring Doubt","Echoing Loneliness","Hissing Disgust","Buzzing Anxiety","Crackling Tension","Popping Curiosity","Howling Despair","Splashing Playfulness","Tinkling Amusement","Booming Confidence","Dripping Melancholy","Rustling Restlessness","Beating Passion","Screeching Shock","Fluttering Flirtation","Snapping Irritation","Groaning Exhaustion","Whistling Freedom","Purring Satisfaction","Banging Desperation","Creaking Anticipation","Sizzling Excitement","Thudding Dread","Jangling Nervousness","Drumming Eagerness","Crashing Overwhelm","Giggling Mirth","Rumbling Power","Vibrating Energy","Stomping Stubbornness","Spluttering Embarrassment","Sniffling Sentimentality","Blaring Boldness","Flickering Fascination","Crooning Lull","Whining Pettiness","Plopping Disappointment","Trilling Enchantment"]

def create_descriptions(num_of_variations: int, description:str, bpm: str):
  descriptions = []
  for i in range(num_of_variations):
    random_description = random_descriptions[random.randint(0, len(random_descriptions)-1)]
    constructed = description + ", " + random_description + ", high fidelity loop, bpm: " + bpm
    descriptions.append(constructed)

  return descriptions

def length_of_complete_bar_or_beat(bpm, audio_length):
    # Calculate the duration of one beat in seconds
    beat_duration = 60.0 / bpm

    # Calculate the number of beats in the audio length
    total_beats = audio_length / beat_duration

    # If the audio length is less than a bar, round down to the nearest whole beat
    if total_beats < 4:
        length_in_seconds = int(total_beats) * beat_duration
    else:
        # Calculate the number of complete bars in 4/4 time
        complete_bars = int(total_beats // 4)
        # Calculate the length of these complete bars in seconds
        length_in_seconds = complete_bars * 4 * beat_duration

    return length_in_seconds

def calculate_average_bpm(output):
    print("calculate_average_bpm")
    """
    Calculate a refined average BPM from BeatNet output by filtering outliers.

    :param output: numpy array of shape (n, 2) where each row is [time, beat_position]
    :return: Refined average BPM as a float
    """
    # Calculate time differences (beat durations)
    beat_durations = np.diff(output[:, 0])

    # Filter outliers
    Q1 = np.percentile(beat_durations, 25)
    Q3 = np.percentile(beat_durations, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_durations = beat_durations[(beat_durations >= lower_bound) & (beat_durations <= upper_bound)]

    # Calculate average beat duration from filtered data
    if len(filtered_durations) == 0:
        return 0  # Avoid division by zero if there are no beats left after filtering
    average_beat_duration = np.mean(filtered_durations)

    # Convert to BPM
    bpm = 60 / average_beat_duration

    # Round to nearest whole number
    rounded_bpm = round(bpm)

    return rounded_bpm

def extract_bpm(input_file):
    print("extract_bpm")
    # Generate a unique directory name
    unique_dir_name = f"/tmp/{uuid.uuid4()}"
    os.makedirs(unique_dir_name, exist_ok=True)

    # Copy the input file to the new directory
    new_file_path = os.path.join(unique_dir_name, os.path.basename(input_file))
    shutil.copy(input_file, new_file_path)

    # Initialize the estimator
    estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)

    # Use the new directory in estimator.process
    Output = estimator.process(new_file_path)

    bpm = calculate_average_bpm(Output)

    return bpm


@ui_param('num_of_variations', 'RunesNumberSlider', min=0, max=10, step=1, default=1)
async def runes_func(num_of_variations: int = 1, description: str = "describe your sound ..", audio_input: RunesFilePath = None):
    try:
        print("FILEPATH: " + str(audio_input))

        audio_length = utils.get_audio_length(audio_input)
        audio_length = length_of_complete_bar_or_beat(runes.get_daw_bpm(), audio_length)
        print("audio_length: " + str(audio_length))

        bpm_str = str(round(runes.get_daw_bpm()))
        print("bpm: " + bpm_str)

        closest_match = {'file': None, 'deviation': float('inf')}  # Initialize with a large deviation
        generated_files = []  # List to store generated files and their deviations

        while num_of_variations > 0:
            max_bpm_checks = 3
            bpm_checks = 0
            # while True:
            print("BPM check: " + str(bpm_checks))

            output_audio_name = f"{str(uuid.uuid4())}.wav"
            model.set_generation_params(duration=audio_length)
            descriptions = create_descriptions(1, description, bpm_str)
            melody, sr = torchaudio.load(audio_input)
            wav = model.generate_with_chroma(descriptions, melody[None], sr)

            generated_audio = wav[0].cpu().numpy().astype('float32')
            generated_audio = generated_audio.squeeze()

            sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')

            generated_bpm = extract_bpm(output_audio_name)

            # Calculate half and double of the target BPM
            target_bpm = runes.get_daw_bpm()
            half_target_bpm = target_bpm / 2
            double_target_bpm = target_bpm * 2

            # Determine if the generated BPM is closer to half or double of the target BPM
            if abs(generated_bpm - half_target_bpm) < abs(generated_bpm - target_bpm) and abs(generated_bpm - half_target_bpm) < abs(generated_bpm - double_target_bpm):
                print("Generated BPM is closer to half of the target BPM.")
                target_bpm = half_target_bpm
            elif abs(generated_bpm - double_target_bpm) < abs(generated_bpm - target_bpm):
                print("Generated BPM is closer to double of the target BPM.")
                target_bpm = double_target_bpm
            else:
                print("Generated BPM is closer to the original target BPM.")

            print("------------------------------------")
            print(f"Input BPM: {bpm_str}")
            print("Generated BPM: " + str(generated_bpm))
            print("Adjusted target BPM for stretching: " + str(target_bpm))
            print("------------------------------------")

            # ############ START - TIME STRETCH ################################
            # Load the audio file
            y, sr = sf.read(output_audio_name)

            # Calculate the stretch factor
            stretch_factor = generated_bpm / target_bpm

            # Apply time stretching
            y_stretched = pyrb.time_stretch(y, sr, stretch_factor)

            # Save the result
            final_output_path = f"{str(uuid.uuid4())}.wav"
            sf.write(final_output_path, y_stretched, sr)
            # ############ END - TIME STRETCH ##################################

            await runes.output().add_file(final_output_path)

            num_of_variations -= 1


        await runes.output().add_message("Returning the input file")
        print("Output sent.")

        return True
    except Exception as e:
        await runes.output().add_error(f"Error in runes_func: {e}")
        return False


runes.set_token(token=runes_token)
runes.set_name("Style Transfer with BPM Detection")
runes.set_description("Provide a text description and an audio file and it will generate and return a clip with closest bpm match.")
runes.register_method(runes_func)

runes.set_input_target_format('wav')
runes.set_input_target_channels(2)
runes.set_input_target_sample_rate(32000)
runes.set_input_target_bit_depth(16)

runes.set_output_target_format('wav')
runes.set_output_target_channels(2)
runes.set_output_target_sample_rate(44100)
runes.set_output_target_bit_depth(16)

runes.connect_to_server()


Looking in indexes: https://download.pytorch.org/whl/cu121


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


ImportError: cannot import name 'MutableSequence' from 'collections' (/usr/lib/python3.10/collections/__init__.py)

REGISTER YOUR TOKEN

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
